In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import logging

sns.set_style("darkgrid")
%matplotlib inline

In [ ]:
sales_df = pd.read_csv('stock_data/stocks_data_2020_till_2020_09_03.csv')
print(len(stocks_df))
stocks_df.head()